In [596]:
import json
import glob
import pandas as pd
import numpy as np
import flat_table
import random

from sklearn.model_selection import train_test_split
from sklearn import metrics


path = './data/road-game-default-rtdb-export_prev_prev.json'
branchs = ['all-games','complete-games']

In [597]:
#load data from the json file
with open(path) as train_file:
    data = json.load(train_file)
    
branch = branchs[0]
df = pd.DataFrame.from_dict(data[branch], orient='index')
df.reset_index(level=0, inplace=True)
df

,index,additional_comments,aggressively_value,behavior,birthYear,blueScore,computer_value,drivinglicense,education,gender,generously_value,log,press-the-read-instructions-button,redScore,steps,win,wisely_value,predictable_value,answer-the-quiz
0,-MZiWIwdnrn8TJ034z2w,,5.00,carefulBehavior,1961,20.00,5.00,valid,BA,male,6.00,"[None, {'blue': 'from: a6, action: left', 'red...",True,23.00,7.00,True,1.00,NaN,NaN
1,-MZiWUxZs8BvWglGhc9r,,2.00,carefulBehavior,1982,19.00,6.00,valid,BA,female,4.00,"[None, {'blue': 'from: a6, action: left', 'red...",True,22.00,8.00,True,4.00,NaN,NaN
2,-MZiWYgxyToIiCSDQY8S,,5.00,aggressiveBehavior,1953,-105.00,7.00,valid,BA,female,2.00,"[None, {'blue': 'from: a6, action: left', 'red...",True,-105.00,5.00,False,6.00,NaN,NaN
3,-MZiWYz_z_BcQlpReXf_,,6.00,aggressiveBehavior,1990,25.00,7.00,valid,Master's degree,male,2.00,"[None, {'blue': 'from: a6, action: left', 'red...",True,21.00,9.00,True,2.00,NaN,NaN
4,-MZiWaNZ6n3K3AZDfr-q,,6.00,aggressiveBehavior,1953,-105.00,6.00,valid,BA,female,2.00,"[None, {'blue': 'from: a6, action: left', 'red...",True,-105.00,5.00,False,1.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,-MbcRp52SnKf2_aDNt2a,NaN,NaN,valueIterationBasedBehavior_v2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,-MbcURCQaLlJFHGxAvyW,NaN,NaN,valueIterationBasedBehavior_v2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,-MbcbjkKh8M10JRrqfwb,NaN,NaN,valueIterationBasedBehavior_v2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
670,-MbcfQpUktm_lYEmxpz2,NaN,NaN,valueIterationBasedBehavior_v2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [598]:
df.columns

Index(['index', 'additional_comments', 'aggressively_value', 'behavior',
       'birthYear', 'blueScore', 'computer_value', 'drivinglicense',
       'education', 'gender', 'generously_value', 'log',
       'press-the-read-instructions-button', 'redScore', 'steps', 'win',
       'wisely_value', 'predictable_value', 'answer-the-quiz'],
      dtype='object')

## Survay results:

upload Id workers and search for duplicate workers

In [599]:
path = './data/workers' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    fromOne = pd.read_csv(filename, index_col=None, header=0)
    fromOne['filename'] = filename
    li.append(fromOne)

workers = pd.concat(li, axis=0, ignore_index=True)

In [600]:
df['WorkerId'] = workers['WorkerId']
df['filename'] = workers['filename'].apply(lambda x: x.split('./data/workers/Batch_')[1].split('_batch_results.csv')[0])

In [601]:
pd.concat(g for _, g in df.groupby('WorkerId') if len(g) > 1).groupby('WorkerId')['index'].count()

WorkerId
A17AF42SNQNH9C    8
A1AZLJ3R5FQ9B7    2
A1BQI9JKLVCIL0    2
A1GC6CN3F0IHWN    2
A1IL81UMEC9OJI    2
                 ..
APB33P7IQEEWK     2
AREQ43Q0T8ENT     2
ASNMT3I94HZ44     2
AT6ICEVY8RM8D     4
ATJ92FYQX176Z     7
Name: index, Length: 65, dtype: int64

how many to drop?

In [602]:
sum(pd.concat(g for _, g in df.groupby('WorkerId') if len(g) > 1).groupby('WorkerId')['index'].count()) - len(pd.concat(g for _, g in df.groupby('WorkerId') if len(g) > 1).groupby('WorkerId')['index'].count())

98

We will keep the first one of any 'WorkerId' instance to keep samples unique

In [603]:
df = df[(~df.duplicated('WorkerId')) | (df['WorkerId'].isnull())]

In [604]:
df.groupby('behavior').mean()

,aggressively_value,blueScore,computer_value,generously_value,redScore,steps,wisely_value,predictable_value
behavior,,,,,,,,
aggressiveBehavior,5.00,-22.38,5.96,3.30,-24.17,6.49,4.33,4.92
carefulBehavior,2.89,10.59,5.15,5.04,13.07,7.61,3.96,NaN
randomBehavior,3.51,-73.79,5.63,3.99,-73.12,12.06,3.74,3.51
semiAggressiveBehavior,4.55,-61.81,5.74,3.09,-62.98,6.38,4.28,4.63
semiCarefulBehavior,3.87,-3.02,5.71,4.31,-1.27,11.80,4.91,4.32
valueIterationBasedBehavior_v2,4.89,-3.68,6.10,3.52,-5.60,8.61,4.77,4.86


In [605]:
df = df[df['behavior'] != 'valueIterationBasedBehavior_v2']

# Statistics of the red agent movements:

### Create the statistics table from the log column: 

In [606]:
class OutOfBoard(Exception):
    pass

In [607]:
def get_pos_after_action(s, a):
    if a == 'U' or a == "up":
        if(s[0] == 'a'):
            raise OutOfBoard("An error occurred, state: "+ s + " action: " + a)
        return "a" + s[1]
    if a == 'D' or a == "down":
        if(s[0] == 'b'):
            raise OutOfBoard("An error occurred, state: "+ s + " action: " + a)
        return "b" + s[1]
    if a == 'L' or a == "left":
        if(s[0] == 'b' or s[1] == '1'):
            raise OutOfBoard("An error occurred, state: "+ s + " action: " + a)
        return "a" + str(int(s[1])-1)
    if a == 'R' or a == "right":
        if(s[0] == 'b' or s[1] == '6'):
            raise OutOfBoard("An error occurred, state: "+ s + " action: " + a)
        return "a" + str(int(s[1])+1)
    if a == 'S' or a == "stay":
        return s

In [608]:
def get_next_state(c_s):
    n_s = c_s.split()
    blueAction = n_s[3]
    blueState = n_s[2]
    redAction = n_s[1]
    redState = n_s[0]
    
    newPosBlue = get_pos_after_action(blueState, blueAction)
    newPosRed = get_pos_after_action(redState, redAction)
    
    return newPosBlue + ' ' + newPosRed

In [609]:
s_r = df['log'].apply(pd.Series).iloc[:,1:]
s_r.columns = s_r.columns.astype(str)
s_r.set_index( df['index'], inplace=True)
log_red = flat_table.normalize(s_r)
log_red = log_red.dropna(how='all')
log_red.set_index(log_red['index'], inplace=True)
log_red = log_red.drop('index', axis=1)

log_len = int(len(log_red.columns)/2)+1
for i in range(1,log_len):
    log_red[str(i)] = log_red[str(i)+'.red'] + ' ' + log_red[str(i)+'.blue']
    log_red = log_red.drop([str(i)+'.red', str(i)+'.blue'],axis=1)
    for substr in ['from: ', ', action:']:
        log_red[str(i)] = log_red[str(i)].apply(lambda x: x.replace(substr, '', 2) if type(x) != float else x)
    
log_len = len(log_red.columns)
for i in range(1,log_len):
    log_red[str(i) + '&' + str(i+1)] = log_red[str(i)] + '&' + log_red[str(i+1)]
    log_red[str(i) + '&' + str(i+1)] = log_red[str(i) + '&' + str(i+1)].apply(lambda x: x.rsplit(' ', 2)[0] if type(x) != float else x)
log_red = log_red.drop([str(i) for i in range(1,log_len+1)],axis=1)

log_red_copy = log_red.copy(deep=True)

log_red = pd.DataFrame(log_red.to_numpy().flatten(), columns=['state'])
log_red = log_red.dropna()

log_red['action'] = log_red['state'].apply(lambda x: x.rsplit('&', 1)[1].rsplit(' ', 1)[1])
log_red['state'] = log_red['state'].apply(lambda x: x.rsplit('&', 1)[0])

log_red['nxt'] = log_red['state'].apply(get_next_state)
log_red['state'] = log_red['state'].str.split().apply(reversed).apply(' '.join)

log_red['value']=1

for a in ['right', 'stay', 'up', 'down', 'left']:
    log_red['state'] = log_red['state'].apply(lambda x: x.replace(a+' ', '', 2))

log_red['state'] = log_red['state'] + ' nxt: ' + log_red['nxt']
log_red = log_red.drop('nxt',axis=1)

log_red = log_red.groupby(['state', 'action']).count()
log_red.reset_index(inplace=True)
log_red = log_red.set_index('state')

In [610]:
result = pd.DataFrame(index=log_red.index.drop_duplicates(), columns=['right', 'stay', 'up', 'down'])
# result.reset_index(level=0, inplace=True)
for action in ['right', 'stay', 'up', 'down']:
    result[action] = log_red[log_red['action'] == action]['value']
result = result.fillna(0)
result.reset_index(inplace=True)
result.sort_index()

,state,right,stay,up,down
0,a1 a2 nxt: a1 a3,10.00,0.00,0.00,0.00
1,a1 a3 nxt: a1 a3,0.00,5.00,0.00,1.00
2,a1 a3 nxt: a1 a4,49.00,0.00,0.00,0.00
3,a1 a3 nxt: a1 b3,0.00,0.00,2.00,0.00
4,a1 a4 nxt: a1 a5,68.00,0.00,0.00,0.00
...,...,...,...,...,...
252,b6 b1 nxt: b6 a1,2.00,0.00,0.00,0.00
253,b6 b1 nxt: b6 b1,0.00,2.00,0.00,0.00
254,b6 b4 nxt: a6 a4,1.00,0.00,0.00,0.00
255,b6 b5 nxt: b6 a5,23.00,3.00,0.00,4.00


In [611]:
# laplace smoving:
for i in range(1,7):
    result.loc[result['state'].str.endswith('a'+str(i)), ['stay', 'right', 'down']] += 1
    
for i in range(1,7):
    result.loc[result['state'].str.endswith('b'+str(i)), ['stay', 'up']] += 1

result = result.set_index('state')

#### Convert it to Probability table:

In [612]:
pd.options.display.float_format = "{:,.2f}".format
result_probs = result.div(result.sum(axis=1), axis=0)

### model accuracy:

In [613]:
def predict(state):
    score = result_probs.loc[state ,:]
    dict_scores = dict(enumerate(score))
    action = random.choices(list(dict_scores.keys()), weights=list(dict_scores.values()))[0]
    return action

def predicts(states):
    y_pred = []
    for s in states:
        y_pred.append(predict(s))
    return y_pred

In [614]:
target = []
for s in result_probs.index:
    target.append( np.argmax(result_probs.loc[s ,:]))

X_train,X_test,y_train,y_test = train_test_split(result_probs.index, target,test_size=0.20,random_state=43, shuffle=True)

In [615]:
k_fold = 1000
scores = []

for _ in range(k_fold):

    y_pred = predicts(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    scores.append(accuracy)

accuracy = np.mean(scores)
# correlation = df.blueScore.corr(df.redScore)
correlation = 0.74

accuracy

0.6163846153846153

In [616]:
correlation

0.74

In [617]:
1 - correlation/accuracy

-0.20054910769998768